### SCRIPT COMPLETO, FALTA SOMENTE COLOCAR AS 25 PERGUNTAS

In [ ]:
# --- CÉLULA ÚNICA: SOLUÇÃO HÍBRIDA ADAPTADA (COM GUIA E AVALIAÇÃO ESTRUTURADA) ---

# --- 1. INSTALAÇÕES E IMPORTAÇÕES ---
print("Instalando bibliotecas necessárias...")
!pip install openai openpyxl PyMuPDF sentence-transformers numpy tiktoken -q
print("Instalação concluída.")

import os
import time
import io
import json
import base64
import fitz  # PyMuPDF
from google.colab import drive, userdata
import openai
from openpyxl import Workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from sentence_transformers import util
import numpy as np
import tiktoken
from IPython.display import HTML, display
print("Bibliotecas importadas com sucesso!")

# --- 2. CONFIGURAÇÃO DO AMBIENTE ---
try:
    print("\nConectando ao Google Drive...")
    drive.mount('/content/drive')
    print("Google Drive conectado.")
    API_KEY = userdata.get('OPENAI_API_KEY')
    if not API_KEY: raise ValueError("API Key não encontrada nos Segredos do Colab.")
    client = openai.OpenAI(api_key=API_KEY)
    print("Cliente OpenAI configurado com sucesso.")
except Exception as e:
    print(f"ERRO na configuração: {e}")
    raise SystemExit("Execução interrompida.")

# --- 3. DEFINIÇÕES DE PERGUNTAS E PROMPTS (COMPLETO) ---
perguntas = [
    "Qual órgão do corpo humano é responsável por filtrar o sangue e produzir a urina?", "Por quanto tempo um homem pode ter incontinência urinária após uma cirurgia de retirada da próstata?", "Quais são os principais fatores de risco para a disfunção erétil?", "Como prevenir a incontinência urinária masculina?", "Qual é a função da bexiga urinária?", "Quais são os três principais tipos de incontinência urinária masculina?", "Qual prática complementar foi estudada para auxiliar no controle da incontinência urinária pós-prostatectomia radical?", "Ao receberem alta hospitalar, o que os pacientes submetidos à cirurgia, em sua grande maioria necessitam fazer?", "Qual tecnologia foi desenvolvida para auxiliar no tratamento da incontinência urinária pós-prostatectomia?", "Qual instrumento foi adaptado para avaliar a disfunção sexual em homens brasileiros com sintomas do trato urinário inferior?", "Segurar muito o xixi pode causar algum problema?", "Tem algum remédio que ajuda a não perder urina?", "Fazer exercício para o xixi parar de escapar funciona mesmo?", "Fazer muita força para urinar é normal?", "Incontinência tem cura ou vou ter isso pra sempre?", "Tomar muita água piora ou melhora o problema de xixi solto?", "O que posso fazer para não ter que levantar à noite toda hora pra urinar?", "Por que tenho vontade de urinar toda hora?", "O que posso fazer para não ter escapes de urina durante o dia?", "Beber cerveja piora a incontinência urinária?", "Fiz a cirurgia recentemente. Agora faço xixi sem querer quando dou risada ou espirro. Isso é normal?", "Por que às vezes meu pênis não fica duro como antes?", "Depois da cirurgia de próstata, ainda vou conseguir ter ereção?", "Ficar muito tempo sem fazer sexo pode causar impotência?", "Tenho evitado fazer sexo depois da cirurgia. Isso é bom?"
]

prompt_template_heuristico = """### **[INÍCIO DO PROMPT MESTRE]**

**Seção 1: Definição da Persona e Missão**

O assistente virtual operará como um modelo de linguagem especializado em recuperação pós-prostatectomia, saúde da próstata e incontinência urinária masculina. O seu objetivo é fornecer respostas precisas, relevantes, abrangentes e expressas com clareza. O assistente atua como uma fonte de informação para apoiar pacientes que passaram por prostatectomia radical.

**Seção 2: Princípio de Fidelidade ao Contexto**

A sua operação é governada pela diretriz da utilização exclusiva das informações contidas no `CONTEXTO RELEVANTE`. O assistente não deve consultar conhecimento externo, a sua base de dados de treinamento ou outras fontes de informação. Este princípio visa garantir a consistência e a fidelidade ao material de origem, que funciona como a fonte de verdade para a tarefa.

**Seção 5: Formato da Resposta Final**

A resposta final não deve conter menção às heurísticas, aos passos do raciocínio ou a qualquer metadado do processo. A resposta deve ser apresentada com base no conhecimento fornecido.

O resultado deve ser:

* Uma resposta textual coesa, adaptada para pacientes que passaram por prostatectomia e clara.
* Redigida em português com no máximo 5 linhas para a resposta.
* A resposta deve ser direta sem apresentar os detalhes de sua construção, seção, passos, etc.
* **Citação da Fonte:** Ao final da resposta, incluir uma seção de "Referências" listando o nome do documento e o número da página de onde a informação foi extraída.

**Seção 6: Execução da Tarefa**

Com base no protocolo descrito, utilizar o `CONTEXTO RELEVANTE` para responder à `PERGUNTA`.

**CONTEXTO RELEVANTE:**
{contexto_relevante}

**PERGUNTA:**
{pergunta_aqui}"""
prompt_template_cot = """### **[INÍCIO DO PROMPT MESTRE]**

**Seção 1: Definição da Persona e Missão**

O assistente virtual operará como um modelo de linguagem especializado em recuperação pós-prostatectomia, saúde da próstata e incontinência urinária masculina. O seu objetivo é fornecer respostas precisas, relevantes, abrangentes e expressas com clareza. O assistente atua como uma fonte de informação para apoiar pacientes que passaram por prostatectomia radical.

**Seção 2: Princípio de Fidelidade ao Contexto**

A sua operação é governada pela diretriz da utilização exclusiva das informações contidas no `CONTEXTO RELEVANTE`. O assistente não deve consultar conhecimento externo, a sua base de dados de treinamento ou outras fontes de informação. Este princípio visa garantir a consistência e a fidelidade ao material de origem, que funciona como a fonte de verdade para a tarefa.

**Seção 5: Formato da Resposta Final**

A resposta final deve ser apresentada como um texto coeso que demonstre a Cadeia de Pensamento.

O resultado deve incluir:

* Uma resposta textual que apresente o percurso lógico até à conclusão.
* Redigida em português com no máximo 5 linhas para a resposta.
* Redação em português.
* **Citação da Fonte:** Ao final da resposta, deve ser incluída uma seção de "Referências", listando o nome do documento (ex: livro, cartilha, apostila) e o número da página de onde a informação foi extraída.

**Seção 6: Execução da Tarefa**

Com base no protocolo descrito, deve ser utilizado o `CONTEXTO RELEVANTE` para responder à `PERGUNTA`.

**CONTEXTO RELEVANTE:**
{contexto_relevante}

**PERGUNTA:**
{pergunta_aqui}"""
prompt_template_direto = """Instrução: Responda à PERGUNTA abaixo utilizando exclusivamente as informações fornecidas no CONTEXTO RELEVANTE. A resposta deve ser direta, objetiva e formulada em português. Não utilize conhecimento externo. Ao final da resposta, inclua uma seção de "Referências" listando o nome do documento e o número da página de onde a informação principal foi extraída.

---

CONTEXTO RELEVANTE:
{contexto_relevante}

---

PERGUNTA:
{pergunta_aqui}"""

# --- 4. FUNÇÕES DO SISTEMA RAG HÍBRIDO ---

def criar_ou_carregar_base_conhecimento(caminho_pasta, client, force_recreate=False):
    caminho_cache = '/content/drive/MyDrive/rag_cache_openai'
    caminho_chunks = os.path.join(caminho_cache, 'chunks_data.json')
    caminho_embeddings = os.path.join(caminho_cache, 'embeddings_openai.npy')
    os.makedirs(caminho_cache, exist_ok=True)

    if not force_recreate and os.path.exists(caminho_chunks) and os.path.exists(caminho_embeddings):
        print("\nCarregando base de conhecimento do cache do Google Drive...")
        with open(caminho_chunks, 'r', encoding='utf-8') as f:
            chunks_com_metadata = json.load(f)
        chunk_embeddings = np.load(caminho_embeddings)
        print(f"Base de conhecimento com {len(chunks_com_metadata)} chunks carregada com sucesso.")
        return chunks_com_metadata, chunk_embeddings

    print("\nCriando nova base de conhecimento (isso pode levar alguns minutos e terá custos de API)...")
    chunks_com_metadata = []
    arquivos_pdf = [f for f in os.listdir(caminho_pasta) if f.lower().endswith('.pdf')]
    print(f"Encontrados {len(arquivos_pdf)} arquivos PDF para processar.")

    for nome_arquivo in arquivos_pdf:
        caminho_completo = os.path.join(caminho_pasta, nome_arquivo)
        try:
            with fitz.open(caminho_completo) as doc:
                for page_num, page in enumerate(doc):
                    text = page.get_text("text")
                    paragrafos = [p.strip() for p in text.split('\n\n') if len(p.strip()) > 150]
                    for p in paragrafos:
                        chunks_com_metadata.append({"texto": p, "fonte": nome_arquivo, "pagina": page_num + 1})
            print(f"   - Processado '{nome_arquivo}' com sucesso.")
        except Exception as e:
            print(f"   - ERRO ao processar '{nome_arquivo}': {e}")

    if not chunks_com_metadata:
        print("ERRO: Nenhum chunk de texto utilizável foi criado.")
        return None, None

    print(f"\nTotal de {len(chunks_com_metadata)} chunks de texto criados.")

    print("Gerando embeddings em lotes com a API da OpenAI...")
    all_embeddings = []
    embedding_model = "text-embedding-3-small"
    tokenizer = tiktoken.encoding_for_model(embedding_model)
    TOKEN_LIMIT_PER_REQUEST = 250000
    current_batch_texts = []
    current_batch_tokens = 0
    try:
        for chunk in chunks_com_metadata:
            text = chunk['texto']
            token_count = len(tokenizer.encode(text))
            if current_batch_tokens + token_count > TOKEN_LIMIT_PER_REQUEST:
                print(f"   - Processando lote com {len(current_batch_texts)} chunks e {current_batch_tokens} tokens...")
                response = client.embeddings.create(input=current_batch_texts, model=embedding_model)
                all_embeddings.extend([item.embedding for item in response.data])
                current_batch_texts = [text]
                current_batch_tokens = token_count
                time.sleep(1)
            else:
                current_batch_texts.append(text)
                current_batch_tokens += token_count
        if current_batch_texts:
            print(f"   - Processando o último lote com {len(current_batch_texts)} chunks e {current_batch_tokens} tokens...")
            response = client.embeddings.create(input=current_batch_texts, model=embedding_model)
            all_embeddings.extend([item.embedding for item in response.data])
        chunk_embeddings = np.array(all_embeddings)
        print("Geração de todos os embeddings concluída.")
    except Exception as e:
        print(f"ERRO CRÍTICO ao gerar embeddings: {e}")
        return None, None

    print("Salvando base de conhecimento no cache do Google Drive...")
    with open(caminho_chunks, 'w', encoding='utf-8') as f:
        json.dump(chunks_com_metadata, f, ensure_ascii=False, indent=4)
    np.save(caminho_embeddings, chunk_embeddings)
    print("Base de conhecimento criada e salva com sucesso.")
    return chunks_com_metadata, chunk_embeddings

def buscar_contexto_relevante(query, chunks_data, chunk_embeddings, client, top_k=8):
    query_embedding = client.embeddings.create(input=[query], model="text-embedding-3-small").data[0].embedding
    query_embedding_np = np.array([query_embedding], dtype=np.float32)
    chunk_embeddings_np = np.array(chunk_embeddings, dtype=np.float32)
    cos_scores = util.pytorch_cos_sim(query_embedding_np, chunk_embeddings_np)[0]
    top_results_indices = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    contexto_formatado = ""
    for i in top_results_indices:
        info = chunks_data[i]
        contexto_formatado += f"---\nFonte: {info['fonte']}, Página: {info['pagina']}\nConteúdo: {info['texto']}\n"
    return contexto_formatado

def gerar_resposta_gpt4(prompt_final, client):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt_final}],
            temperature=0.4
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"   - ERRO ao chamar a API de Chat: {e}")
        return f"[ERRO AO CHAMAR API: {e}]"

# --- FUNÇÃO PRINCIPAL DE CRIAÇÃO DA PLANILHA (CORRIGIDA) ---

def criar_planilha_final(perguntas_lista, respostas_dict):
    """Cria a planilha Excel final, com validação de dados restrita e opções objetivas de classificação."""
    print("\nPASSO FINAL: Criando a planilha Excel com validação restrita...")
    wb = Workbook()

    # --- ABA 1: GUIA DE AVALIAÇÃO (ATUALIZADA) ---
    ws_guia = wb.active
    ws_guia.title = "Guia de Avaliação"

    # Estilos
    title_font = Font(bold=True, size=16, color="FFFFFF")
    title_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="1F497D", end_color="1F497D", fill_type="solid")
    section_font = Font(bold=True, size=14)

    # Título Principal e Objetivo
    ws_guia.merge_cells('A1:B1')
    ws_guia['A1'].value = "Guia para Avaliação das Respostas do Modelo de Linguagem"
    ws_guia['A1'].font = title_font; ws_guia['A1'].fill = title_fill; ws_guia['A1'].alignment = Alignment(horizontal='center', vertical='center')
    ws_guia.row_dimensions[1].height = 30
    ws_guia.merge_cells('A2:B2')
    ws_guia['A2'].value = "Objetivo: Avaliar a qualidade e a eficácia de respostas geradas por Inteligência Artificial. O processo é dividido em 2 etapas para cada pergunta principal."
    ws_guia['A2'].alignment = Alignment(wrap_text=True, vertical='top'); ws_guia.row_dimensions[2].height = 40

    # Etapa 1
    ws_guia.merge_cells('A4:B4'); ws_guia['A4'].value = "Etapa 1: Avaliação de cada Resposta Individual"; ws_guia['A4'].font = section_font; ws_guia['A4'].alignment = Alignment(vertical='center'); ws_guia.row_dimensions[4].height = 22
    ws_guia['A5'] = "Pergunta para a Avaliadora"; ws_guia['A5'].font = header_font; ws_guia['A5'].fill = header_fill
    ws_guia['B5'] = "Opções de Resposta (Menu de Seleção)"; ws_guia['B5'].font = header_font; ws_guia['B5'].fill = header_fill
    questoes_etapa1 = [
        ("1. Esta resposta está correta?", "Totalmente / Em parte / Não"),
        ("2. A resposta resolve a dúvida do paciente por completo?", "Resolve tudo / Resolve em parte / Não resolve"),
        ("3. A linguagem é fácil para um paciente entender?", "Muito fácil / Razoavelmente Fácil / Resposta Confusa / Resposta Técnica"),
        ("4. A resposta cita a fonte da informação?", "Citou a fonte e a página / Citou apenas a fonte / Não citou a fonte / Citou a fonte errada"),
        ("5. A resposta possui alguma informação inventada?", "Sim / Não"),
        ("6. Observações sobre esta resposta específica:", "(Campo de texto livre)")
    ]
    for i, (q, r) in enumerate(questoes_etapa1, 6):
        ws_guia[f'A{i}'] = q; ws_guia[f'B{i}'] = r; ws_guia.row_dimensions[i].height = 25
        ws_guia[f'A{i}'].alignment = Alignment(vertical='center', wrap_text=True); ws_guia[f'B{i}'].alignment = Alignment(vertical='center', wrap_text=True)

    # Etapa 2 (com texto da pergunta 3 atualizado para refletir o menu de seleção)
    ws_guia.merge_cells('A12:B12')
    ws_guia.merge_cells('A13:B13'); ws_guia['A13'].value = "Etapa 2: Síntese Final (após avaliar as 3 respostas de uma pergunta)"; ws_guia['A13'].font = section_font; ws_guia['A13'].alignment = Alignment(vertical='center'); ws_guia.row_dimensions[13].height = 22
    ws_guia['A14'] = "Pergunta para a Avaliadora"; ws_guia['A14'].font = header_font; ws_guia['A14'].fill = header_fill
    ws_guia['B14'] = "Opções de Resposta"; ws_guia['B14'].font = header_font; ws_guia['B14'].fill = header_fill
    questoes_etapa2 = [
        ("1. No geral, qual o prompt ?", "Prompt Heurístico / Prompt CoT / Prompt Direto"),
        ("2. Justifique a resposta anterior.", "(Campo de texto livre)"),
        ("3. Classifique os 3 prompts do melhor para a pior:", "(Menu de seleção com 6 ordens possíveis)") # Texto atualizado
    ]
    for i, (q, r) in enumerate(questoes_etapa2, 15):
        ws_guia[f'A{i}'] = q; ws_guia[f'B{i}'] = r; ws_guia.row_dimensions[i].height = 25
        ws_guia[f'A{i}'].alignment = Alignment(vertical='center', wrap_text=True); ws_guia[f'B{i}'].alignment = Alignment(vertical='center', wrap_text=True)

    ws_guia.column_dimensions['A'].width = 55; ws_guia.column_dimensions['B'].width = 65

    # --- ABA 2: AVALIAÇÃO DAS RESPOSTAS (COM VALIDAÇÃO RESTRITA) ---
    ws = wb.create_sheet("Avaliação das Respostas")
    header = [
        "ID Pergunta", "Pergunta / Abordagem", "Resposta Gerada", "1. Esta resposta está correta?",
        "2. A resposta resolve a dúvida do paciente por completo?", "3. A linguagem é fácil para um paciente entender?",
        "4. A resposta cita a fonte da informação?", "5. A resposta possui alguma informação inventada?",
        "6. Observações sobre esta resposta específica:"
    ]
    ws.append(header)

    # Estilos
    header_font_eval = Font(bold=True, color="FFFFFF"); header_fill_eval = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
    question_fill = PatternFill(start_color="DCE6F1", end_color="DCE6F1", fill_type="solid")
    sintese_fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
    sintese_header_fill = PatternFill(start_color="BFBFBF", end_color="BFBFBF", fill_type="solid")
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    for cell in ws[1]: cell.font = header_font_eval; cell.fill = header_fill_eval; cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

    # --- INÍCIO DA ALTERAÇÃO: Função para criar DataValidation RESTRITA ---
    def create_restricted_dv(formula):
        dv = DataValidation(type="list", formula1=formula, allow_blank=True)
        dv.error = 'Por favor, selecione um valor da lista.'
        dv.errorTitle = 'Entrada Inválida'
        dv.prompt = 'Selecione uma opção da lista.'
        dv.promptTitle = 'Seleção de Opção'
        dv.showErrorMessage = True
        dv.errorStyle = 'stop' # A MUDANÇA CHAVE: Impede digitação de valores fora da lista
        return dv
    # --- FIM DA ALTERAÇÃO ---

    dv1 = create_restricted_dv('"Totalmente,Em parte,Não"')
    dv2 = create_restricted_dv('"Resolve tudo,Resolve em parte,Não resolve"')
    dv3 = create_restricted_dv('"Muito fácil,Razoavelmente Fácil, Confusa, Resposta técnica"')
    dv4 = create_restricted_dv('"Cita a fonte e a página,Citou apenas a fonte,Não citou a fonte, Citou a fonte errada"')
    dv5 = create_restricted_dv('"Sim,Não"')
    dv_sintese_1 = create_restricted_dv('"Prompt Heurístico,Prompt CoT,Prompt Direto"')

    # --- INÍCIO DA ALTERAÇÃO: Lista de opções para a pergunta de classificação ---
    classificacao_options = [
        "1º Heurístico > 2º CoT > 3º Direto", "1º Heurístico > 2º Direto > 3º CoT",
        "1º CoT > 2º Heurístico > 3º Direto", "1º CoT > 2º Direto > 3º Heurístico",
        "1º Direto > 2º Heurístico > 3º CoT", "1º Direto > 2º CoT > 3º Heurístico"
    ]
    # Converte a lista Python para o formato de string que o Excel espera: "Opção 1,Opção 2,..."
    dv_sintese_3 = create_restricted_dv(f'"{",".join(classificacao_options)}"')
    # --- FIM DA ALTERAÇÃO ---

    ws.add_data_validation(dv1); ws.add_data_validation(dv2); ws.add_data_validation(dv3)
    ws.add_data_validation(dv4); ws.add_data_validation(dv5); ws.add_data_validation(dv_sintese_1)
    ws.add_data_validation(dv_sintese_3) # Adiciona a nova validação à planilha

    # Preenchimento
    current_row = 2
    prompt_types = ["Prompt Heurístico", "Prompt CoT", "Prompt Direto"]
    for i, pergunta_texto in enumerate(perguntas_lista, 1):
        start_merge_row = current_row
        ws.cell(row=current_row, column=1, value=i)
        ws.cell(row=current_row, column=2, value=pergunta_texto)
        for col in range(1, len(header) + 1): ws.cell(row=current_row, column=col).fill = question_fill; ws.cell(row=current_row, column=col).font = Font(bold=True)
        current_row += 1

        for tipo in prompt_types:
            id_resposta = f"{i}-{tipo}"; ws.cell(row=current_row, column=2, value=f"Abordagem: {tipo}")
            ws.cell(row=current_row, column=3, value=respostas_dict.get(id_resposta, "[RESPOSTA NÃO ENCONTRADA]"))
            for col in range(2, len(header) + 2): ws.cell(row=current_row, column=col).border = thin_border
            dv1.add(ws[f'D{current_row}']); dv2.add(ws[f'E{current_row}']); dv3.add(ws[f'F{current_row}'])
            dv4.add(ws[f'G{current_row}']); dv5.add(ws[f'H{current_row}'])
            current_row += 1

        ws.merge_cells(start_row=current_row, start_column=2, end_row=current_row, end_column=len(header))
        cell_sintese = ws.cell(row=current_row, column=2, value="Etapa 2: Síntese Final")
        cell_sintese.fill = sintese_header_fill; cell_sintese.font = Font(bold=True); cell_sintese.alignment = Alignment(horizontal='center')
        current_row += 1

        # --- INÍCIO DA ALTERAÇÃO: Lógica para aplicar a validação correta a cada pergunta de síntese ---
        sintese_rows_config = [
            ("1. No geral, qual o prompt ?", 'geral'),
            ("2. Justifique a resposta anterior.", 'livre'), # 'livre' significa sem validação
            ("3. Classifique os 3 prompts do melhor para a pior:", 'classificacao')
        ]
        for texto_sintese, tipo_dv in sintese_rows_config:
            ws.cell(row=current_row, column=2, value=texto_sintese).fill = sintese_fill
            ws.merge_cells(start_row=current_row, start_column=3, end_row=current_row, end_column=len(header))
            cell_to_fill = ws.cell(row=current_row, column=3); cell_to_fill.fill = sintese_fill
            # Aplica a validação de dados correta com base no tipo
            if tipo_dv == 'geral':
                dv_sintese_1.add(cell_to_fill)
            elif tipo_dv == 'classificacao':
                dv_sintese_3.add(cell_to_fill)
            # Se tipo_dv for 'livre', nenhuma validação é adicionada
            current_row += 1
        # --- FIM DA ALTERAÇÃO ---

        ws.merge_cells(start_row=start_merge_row, start_column=1, end_row=current_row - 1, end_column=1)
        ws.cell(row=start_merge_row, column=1).alignment = Alignment(horizontal='center', vertical='center')

    # Ajustes finais da planilha
    ws.column_dimensions['B'].width = 45; ws.column_dimensions['C'].width = 90; ws.column_dimensions['D'].width = 25
    ws.column_dimensions['E'].width = 30; ws.column_dimensions['F'].width = 30; ws.column_dimensions['G'].width = 30
    ws.column_dimensions['H'].width = 25; ws.column_dimensions['I'].width = 60
    for row in ws.iter_rows(min_row=2):
        for cell in row: cell.alignment = Alignment(wrap_text=True, vertical='top')

    ws.freeze_panes = 'A2'

    # Geração do link para download
    virtual_workbook = io.BytesIO(); wb.save(virtual_workbook)
    b64 = base64.b64encode(virtual_workbook.getvalue()).decode('utf-8')
    file_name = "instrumento_avaliacao_FINAL_corrigido.xlsx"
    link_html = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="{file_name}">Clique aqui para baixar a planilha de avaliação (CORRIGIDA)</a>'
    display(HTML(link_html))
    print("\nPlanilha finalizada e pronta para download.")

# --- 5. EXECUÇÃO PRINCIPAL ---
if __name__ == '__main__':
    # ATENÇÃO: Substitua pelo caminho correto no seu Google Drive
    caminho_da_pasta_pdfs = '/content/drive/MyDrive/materiais para referencia/referencias prompt IUProst'

    # Mantenha como False para usar o cache e economizar custos de API, ou True para forçar a releitura dos PDFs
    FORCAR_RECRIAÇÃO = False

    chunks_data, chunk_embeddings = criar_ou_carregar_base_conhecimento(caminho_da_pasta_pdfs, client, force_recreate=FORCAR_RECRIAÇÃO)

    if chunks_data is not None:
        respostas_finais = {}
        total_perguntas = len(perguntas)
        total_geracoes = total_perguntas * 3
        print(f"\nIniciando a geração de {total_geracoes} respostas usando o GPT-4o-mini e a busca de alta qualidade.")

        prompts = {
            "Prompt Heurístico": prompt_template_heuristico,
            "Prompt CoT": prompt_template_cot,
            "Prompt Direto": prompt_template_direto,
        }

        for i, pergunta in enumerate(perguntas, 1):
            print(f"\nProcessando Pergunta {i}/{total_perguntas}: '{pergunta[:60]}...'")
            contexto = buscar_contexto_relevante(pergunta, chunks_data, chunk_embeddings, client)

            for tipo, prompt_template in prompts.items():
                print(f"   - Gerando resposta para o {tipo}...")
                prompt_final = prompt_template.format(contexto_relevante=contexto, pergunta_aqui=pergunta)
                resposta_atual = gerar_resposta_gpt4(prompt_final, client)
                id_resposta = f"{i}-{tipo}"

                print(f"\n   ========= RESPOSTA RECEBIDA ({id_resposta}) =========")
                print(resposta_atual)
                print("   ========================================\n")

                respostas_finais[id_resposta] = resposta_atual
                time.sleep(10) # Pausa para não sobrecarregar a API

        print("\nGeração de todas as respostas concluída!")
        criar_planilha_final(perguntas, respostas_finais)
    else:
        print("\nERRO CRÍTICO: A execução foi interrompida porque a base de conhecimento não pôde ser criada ou carregada.")

Instalando bibliotecas necessárias...
Instalação concluída.
Bibliotecas importadas com sucesso!

Conectando ao Google Drive...
Mounted at /content/drive
Google Drive conectado.
Cliente OpenAI configurado com sucesso.

Carregando base de conhecimento do cache do Google Drive...
Base de conhecimento com 4120 chunks carregada com sucesso.

Iniciando a geração de 3 respostas usando o GPT-4o-mini e a busca de alta qualidade.

Processando Pergunta 1/1: 'Qual órgão do corpo humano é responsável por filtrar o sangu...'
   - Gerando resposta para o Prompt Heurístico...

   ========= RESPOSTA RECEBIDA (1-Prompt Heurístico) =========
Os rins são os órgãos responsáveis por filtrar o sangue e produzir a urina. Eles desempenham um papel fundamental na excreção de resíduos e na regulação do equilíbrio de fluidos e eletrólitos no corpo. 

**Referências:** Livro urologia fundamental-09-09-10.pdf, Página: 19.

   - Gerando resposta para o Prompt CoT...

   ========= RESPOSTA RECEBIDA (1-Prompt CoT) ====


Planilha finalizada e pronta para download.


### OUTRA VERSÃO POIS A ANTERIOR FALTOU ALGUMAS ANÁLISES

In [ ]:
# =================================================================================
# SCRIPT DE ANÁLISE ESTATÍSTICA - GERADOR DE CAPÍTULO DE LATEX (VERSÃO 6.5 - FINAL CORRIGIDO)
# ESTA VERSÃO CORRIGE A LEITURA DE ARQUIVOS .XLSX, REINTRODUZINDO O PARÂMETRO 'sheet_name'.
# =================================================================================

# ------------------------------------------------------------------------------
# PASSO 0: INSTALAÇÃO DE DEPENDÊNCIAS E IMPORTAÇÃO
# ------------------------------------------------------------------------------
# Tenta importar as bibliotecas e, se não existirem, instala-as.
try:
    import pandas as pd
    import numpy as np
    import statsmodels
    import scipy
    import textstat
    import pytz
    from google.colab import files
except ImportError:
    print("Instalando as bibliotecas necessárias (pandas, openpyxl, statsmodels, scipy, textstat, pytz)...")
    import subprocess
    import sys
    # A instalação de openpyxl é necessária para pd.read_excel
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "openpyxl", "statsmodels", "scipy", "textstat", "pytz"])
    print("Instalação concluída. Por favor, execute a célula novamente para carregar os módulos.")
    # Força a interrupção para que o usuário possa reexecutar a célula.
    exit()

from statsmodels.stats.inter_rater import fleiss_kappa
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency, f_oneway, levene
import io
import os
import traceback
from datetime import datetime

# ------------------------------------------------------------------------------
# PASSO 1: UPLOAD E PREPARAÇÃO DOS DADOS
# ------------------------------------------------------------------------------
print("--- PASSO 1: UPLOAD E PREPARAÇÃO DOS DADOS ---")

print("\nPor favor, faça o upload de UM OU MAIS arquivos de avaliação (.xlsx ou .csv).")
uploaded = files.upload()

if not uploaded:
    print("\nERRO: Nenhum arquivo foi enviado. Por favor, execute o script novamente e faça o upload de pelo menos um arquivo.")
else:
    # Bloco principal que executa toda a lógica após o upload
    try:
        # =================================================================
        # 1.1 LEITURA E PROCESSAMENTO DOS DADOS
        # =================================================================
        print(f"\nLendo e processando {len(uploaded)} arquivo(s)...")
        all_processed_dfs = []

        def preprocess_df(df, avaliadora_nome):
            """Prepara e limpa o dataframe de cada avaliadora."""
            df['Avaliadora'] = avaliadora_nome

            # Extrai a abordagem e preenche para as linhas seguintes
            df['Abordagem'] = df['Pergunta / Abordagem'].apply(lambda x: x.split(': ')[1] if isinstance(x, str) and 'Abordagem:' in x else np.nan)
            df['Abordagem'] = df['Abordagem'].ffill()

            # Cria um ID único para cada pergunta para agrupar as 3 respostas
            is_question_row = df['Pergunta / Abordagem'].str.contains('Abordagem:', na=False)
            df['ID_Pergunta'] = is_question_row.cumsum()

            # Renomeia as colunas de avaliação para nomes mais simples
            rename_map = {
                '1. Esta resposta é segura e correta? (Se a resposta é confiável e válida para o paciente)': 'Seguranca e Correcao',
                '2. A resposta resolve a dúvida do paciente por completo?': 'Resolucao da Duvida',
                '3. A linguagem é de fácil entendimento para um paciente?': 'Facilidade de Entendimento',
                '4. A resposta cita a fonte da informação?': 'Citacao da Fonte',
                '5. A resposta possui alguma informação inventada?': 'Informacao Inventada'
            }

            df_clean = df.rename(columns=rename_map)

            # Garante que a coluna 'Resposta Gerada' existe
            if 'Resposta Gerada' not in df_clean.columns:
                df_clean['Resposta Gerada'] = df_clean['Pergunta / Abordagem']

            # Filtra apenas as linhas que contêm avaliações
            if 'Seguranca e Correcao' in df_clean.columns:
                df_clean = df_clean[df_clean['Seguranca e Correcao'].notna()].copy()
            else:
                 raise KeyError("A coluna de avaliação 'Seguranca e Correcao' não foi encontrada. Verifique os nomes das colunas no arquivo.")

            return df_clean

        for i, (filename, content) in enumerate(uploaded.items()):
            print(f"  - Processando '{filename}' como Avaliador(a) {i+1}...")
            file_extension = os.path.splitext(filename)[1]
            try:
                if file_extension == '.xlsx':
                     # CORREÇÃO CRÍTICA: REINSERINDO O PARÂMETRO 'sheet_name'
                     df_raw = pd.read_excel(io.BytesIO(content), sheet_name='Avaliação das Respostas')
                elif file_extension == '.csv':
                    try:
                        df_raw = pd.read_csv(io.BytesIO(content), encoding='utf-8')
                    except UnicodeDecodeError:
                        df_raw = pd.read_csv(io.BytesIO(content), encoding='latin-1')
                else:
                    print(f"    AVISO: Formato de arquivo '{file_extension}' não suportado para '{filename}'. Pulando.")
                    continue

                avaliadora_nome = f"Avaliadora {i+1}"
                processed_df = preprocess_df(df_raw, avaliadora_nome)
                all_processed_dfs.append(processed_df)
            except Exception as e:
                print(f"    ERRO ao processar o arquivo {filename}: {e}")
                print("    Verifique se o arquivo tem a aba 'Avaliação das Respostas' e as colunas corretas.")


        if not all_processed_dfs:
            raise ValueError("Nenhum arquivo de avaliação pôde ser processado. O script será encerrado.")

        df_total_consolidado = pd.concat(all_processed_dfs, ignore_index=True)
        # Normalização dos valores das colunas de critérios
        for col in ['Seguranca e Correcao', 'Resolucao da Duvida', 'Facilidade de Entendimento', 'Citacao da Fonte', 'Informacao Inventada']:
            if col in df_total_consolidado.columns and pd.api.types.is_string_dtype(df_total_consolidado[col]):
                 df_total_consolidado[col] = df_total_consolidado[col].str.strip().str.lower()

        print("Dados processados com sucesso.")

        # =================================================================
        # 1.2 FUNÇÕES DE GERAÇÃO DE LATEX
        # =================================================================
        def escape_latex(text):
            if not isinstance(text, str): return text
            return text.replace('&', '\\&').replace('%', '\\%').replace('$', '\\$').replace('#', '\\#').replace('_', '\\_').replace('{', '\\{').replace('}', '\\}').replace('~', '\\textasciitilde{}').replace('^', '\\textasciicircum{}').replace('\\', '\\textbackslash{}')

        def generate_descriptive_table_latex(df_crosstab, caption, label):
            df_crosstab_copy = df_crosstab.copy()
            df_crosstab_copy.index.name = 'Tipo de Prompt'
            df_crosstab_copy.columns = [escape_latex(str(c).capitalize()) for c in df_crosstab_copy.columns]

            df_total = df_crosstab_copy.sum(axis=1)
            df_percent = df_crosstab_copy.div(df_total, axis=0).fillna(0) * 100

            df_formatted = pd.DataFrame(index=df_crosstab_copy.index)
            for col in df_crosstab_copy.columns:
                df_formatted[col] = df_crosstab_copy[col].astype(str) + " (" + df_percent[col].map('{:.1f}\\%'.format) + ")"

            return df_formatted.reset_index().to_latex(index=False, caption=caption, label=label, position='H', escape=False)

        def generate_pgfplot_bar(contingency_table, title, xlabel, ylabel, criterion_name):
            ct_for_pgf = contingency_table.copy().reset_index()
            clean_cols = [''.join(e for e in str(c) if e.isalnum() or e == '_') for c in ct_for_pgf.columns]
            ct_for_pgf.columns = clean_cols

            pgf_data_str = ct_for_pgf.to_csv(index=False, sep=',')

            safe_name = ''.join(e for e in criterion_name if e.isalnum())
            data_id = f"dados{safe_name}"

            latex_code = [f"\\pgfplotstableread[col sep=comma]{{{pgf_data_str}}}{{\\{data_id}}}"]

            legend_entries = [escape_latex(str(c).capitalize()) for c in contingency_table.columns]
            legend_str = ", ".join(legend_entries)

            x_coord_col_name = ct_for_pgf.columns[0]

            symbolic_coords = [item.replace('_', ' ') for item in contingency_table.index]

            latex_code.extend([
                "\\begin{figure}[H]", "\\centering", "\\begin{tikzpicture}",
                "\\begin{axis}[",
                f"    title={{{escape_latex(title)}}},", "ybar stacked,", "bar width=0.8cm,", f"enlarge x limits={{0.25}},",
                f"    xlabel={{{escape_latex(xlabel)}}},", f"    ylabel={{{escape_latex(ylabel)}}},",
                f"    symbolic x coords={{{', '.join(symbolic_coords)}}},",
                "    xtick=data,", "x tick label style={rotate=45, anchor=east},",
                "    legend style={at={(0.5,1.1)}, anchor=south, legend columns=-1, /tikz/every even column/.append style={column sep=0.5cm}}",
                "]"
            ])
            for col_name in ct_for_pgf.columns[1:]:
                latex_code.append(f"\\addplot table [x={x_coord_col_name}, y={col_name}] {{\\{data_id}}};")
            latex_code.extend([
                f"\\legend{{{legend_str}}}", "\\end{axis}", "\\end{tikzpicture}",
                f"\\caption{{Distribuição das avaliações para o critério “{escape_latex(criterion_name)}”.}}",
                f"\\label{{fig:{safe_name}}}", "\\end{figure}"
            ])
            return "\n".join(latex_code)

        def generate_pgfplot_boxplot(df_scores, value_col, title, xlabel, ylabel, caption, label):
            latex_code = [
                "\\begin{figure}[H]", "\\centering", "\\begin{tikzpicture}",
                "\\begin{axis}[",
                f"    title={{{escape_latex(title)}}},", f"    xlabel={{{escape_latex(xlabel)}}},", f"    ylabel={{{escape_latex(ylabel)}}},",
                "    boxplot/draw direction=y,", "xtick={1,2,3},",
                "    xticklabels={{Prompt CoT}, {Prompt Direto}, {Prompt Heurístico}},",
                "    x tick label style={rotate=45, anchor=east}",
                "]"
            ]
            prompt_order = ['Prompt CoT', 'Prompt Direto', 'Prompt Heurístico']
            for abordagem in prompt_order:
                scores = df_scores[df_scores['Abordagem'] == abordagem][value_col].dropna()
                if not scores.empty:
                    scores_str = ' \\\\ '.join(map(lambda x: f"{x:.2f}", scores))
                    latex_code.append(f"\\addplot+ [boxplot] table [row sep=\\\\, y index=0] {{ data \\\\ {scores_str} \\\\ }};")
            latex_code.extend([
                "\\end{axis}", "\\end{tikzpicture}",
                f"\\caption{{{escape_latex(caption)}}}",
                f"\\label{{fig:{label}}}", "\\end{figure}"
            ])
            return "\n".join(latex_code)

        def generate_tukey_latex(tukey_results, caption, label):
            df_tukey = pd.read_html(tukey_results.summary().as_html(), header=0)[0]
            df_tukey = df_tukey.rename(columns={'p-adj': 'p-ajustado', 'meandiff': 'dif. médias'})
            df_tukey['dif. médias'] = df_tukey['dif. médias'].map('{:.2f}'.format)
            df_tukey['p-ajustado'] = df_tukey['p-ajustado'].map('{:.4f}'.format)
            df_tukey['reject'] = df_tukey['reject'].replace({True: 'Sim', False: 'Não'})
            df_tukey = df_tukey.rename(columns={'reject': 'Rejeita H0 (p<0.05)'})
            df_tukey.columns = [escape_latex(c) for c in df_tukey.columns]
            return df_tukey.to_latex(index=False, caption=caption, label=label, position='H', escape=False)

        # =================================================================
        # 2. INÍCIO DA GERAÇÃO DO RELATÓRIO EM LATEX
        # =================================================================
        print("\nIniciando a geração do relatório em LaTeX...")
        latex_output = []

        tz_br = pytz.timezone('America/Sao_Paulo')
        timestamp = datetime.now(tz_br).strftime('%Y-%m-%d %H:%M:%S')

        latex_output.append(
            fr"""% ===================================================================
% INÍCIO DO CAPÍTULO DE RESULTADOS GERADO AUTOMATICAMENTE
% Gerado em: {timestamp}
% Análise baseada em {len(uploaded)} arquivo(s) de avaliação.
% Versão do Script: 6.5 (Final Corrigido)
% ===================================================================
%
% CERTIFIQUE-SE DE QUE OS SEGUINTES PACOTES ESTÃO NO SEU ARQUIVO .tex PRINCIPAL:
% \usepackage{{booktabs}}
% \usepackage{{float}}
% \usepackage{{tikz}}
% \usepackage{{pgfplots}}
% \pgfplotsset{{compat=1.18}}
%
% ===================================================================

\chapter{{Resultados}}
\label{{chap:resultados}}

Neste capítulo, são apresentados os resultados das análises estatísticas realizadas sobre as avaliações das respostas geradas por diferentes tipos de prompts. A análise aborda a distribuição geral das avaliações, a concordância entre os avaliadores, uma comparação da qualidade das respostas entre os prompts e uma avaliação objetiva da legibilidade dos textos.
"""
        )

        evaluation_criteria = {
            'Seguranca e Correcao': 'Segurança e Correção',
            'Resolucao da Duvida': 'Resolução da Dúvida',
            'Facilidade de Entendimento': 'Facilidade de Entendimento',
            'Citacao da Fonte': 'Citação da Fonte',
            'Informacao Inventada': 'Informação Inventada'
        }

        # =================================================================
        # 2.0 ANÁLISE DESCRITIVA
        # =================================================================
        latex_output.append(r"""
\section{Análise Descritiva das Avaliações}
\label{sec:descritiva}
Inicialmente, foi realizada uma análise descritiva para visualizar a distribuição das avaliações para cada um dos cinco critérios. As tabelas a seguir apresentam a contagem absoluta e o percentual de cada categoria de resposta (N (\%)), agrupadas por tipo de prompt.
""")
        for criterion_key, criterion_name in evaluation_criteria.items():
            try:
                df_filtered = df_total_consolidado.dropna(subset=[criterion_key, 'Abordagem'])
                if df_filtered.empty: continue

                crosstab = pd.crosstab(df_filtered['Abordagem'], df_filtered[criterion_key])
                caption = f"Análise descritiva para o critério “{escape_latex(criterion_name)}”."
                label = f"tab:desc_{criterion_key}"
                latex_output.append(generate_descriptive_table_latex(crosstab, caption, label))
            except Exception as e:
                latex_output.append(f"\n% Não foi possível gerar a análise descritiva para {criterion_name}: {e}\n")


        # =================================================================
        # 2.1 ANÁLISE DE CONCORDÂNCIA (Kappa de Fleiss)
        # =================================================================
        latex_output.append(r"""
\section{Análise de Concordância entre Avaliadores}
\label{sec:concordancia}
""")

        num_avaliadores = len(all_processed_dfs)
        if num_avaliadores < 2:
            latex_output.append("A análise de concordância interavaliador (Kappa de Fleiss) não foi realizada, pois requer no mínimo dois arquivos de avaliação.\n")
        else:
            latex_output.append(
                fr"""
Para quantificar o grau de concordância entre os {num_avaliadores} avaliadores, foi utilizado o coeficiente Kappa de Fleiss. Esta medida corrige a concordância que poderia ocorrer puramente ao acaso. A Tabela \ref{{tab:kappa_fleiss}} apresenta os valores de Kappa calculados. Valores próximos a 1 indicam concordância perfeita, enquanto valores próximos de 0 indicam que a concordância não é melhor do que o acaso.
"""
            )

            kappa_results = {}
            for criterion_key, criterion_name in evaluation_criteria.items():
                try:
                    df_filtered_kappa = df_total_consolidado.dropna(subset=[criterion_key, 'Abordagem']).copy()

                    contingency_matrix = df_filtered_kappa.groupby(['ID_Pergunta', 'Abordagem'])[criterion_key].value_counts().unstack(fill_value=0)
                    contingency_matrix = contingency_matrix[contingency_matrix.sum(axis=1) >= 2]

                    if contingency_matrix.shape[1] < 2:
                        kappa_value = np.nan
                    else:
                        kappa_value = fleiss_kappa(contingency_matrix.values, method='fleiss')
                    kappa_results[criterion_name] = kappa_value
                except Exception:
                    kappa_results[criterion_name] = np.nan

            kappa_df = pd.DataFrame(list(kappa_results.items()), columns=['Critério de Avaliação', 'Valor de Kappa'])
            kappa_df['Valor de Kappa'] = kappa_df['Valor de Kappa'].map(lambda x: f'{x:.4f}' if pd.notna(x) else 'nan')

            latex_output.append(
                kappa_df.to_latex(index=False, caption='Resultados do coeficiente Kappa de Fleiss para cada critério.', label='tab:kappa_fleiss', position='H', column_format='lc', header=[escape_latex(c) for c in kappa_df.columns])
            )

        # =================================================================
        # 2.2 ANÁLISE COMPARATIVA DOS PROMPTS (Qui-Quadrado)
        # =================================================================
        latex_output.append(
            r"""
\section{Análise Comparativa dos Tipos de Prompt}
\label{sec:comparacao_prompts}

Para determinar se havia uma associação estatisticamente significativa entre o tipo de prompt e a avaliação recebida, foi empregado o teste Qui-quadrado de Independência ($\chi^2$). A hipótese nula ($H_0$) é que as variáveis são independentes. Um valor-p inferior a 0.05 indica significância estatística.
"""
        )
        for criterion_key, criterion_name in evaluation_criteria.items():
            latex_output.append(f"\n\\subsection{{{escape_latex(criterion_name)}}}\n")
            try:
                df_filtered_chi2 = df_total_consolidado.dropna(subset=[criterion_key, 'Abordagem']).copy()

                if df_filtered_chi2['Abordagem'].nunique() < 2 or df_filtered_chi2[criterion_key].nunique() < 2:
                    latex_output.append(f"Não foi possível realizar a análise Qui-quadrado para “{escape_latex(criterion_name)}” devido à falta de variação nos dados.")
                    continue

                contingency_table = pd.crosstab(df_filtered_chi2['Abordagem'], df_filtered_chi2[criterion_key])
                chi2, p, dof, _ = chi2_contingency(contingency_table)

                p_value_text = 'menor' if p < 0.05 else 'maior'
                conclusion_text = 'há evidências de uma associação estatisticamente significativa' if p < 0.05 else 'não há evidências de uma associação estatisticamente significativa'

                latex_output.append(f"O teste Qui-quadrado resultou em $\\chi^2({dof}) = {chi2:.4f}$, com um valor-p de ${p:.4f}$. Como o valor-p é {p_value_text} que 0.05, {conclusion_text} entre o tipo de prompt e a avaliação deste critério.")

                latex_output.append(generate_pgfplot_bar(contingency_table, f'Avaliações para “{criterion_name}”', 'Tipo de Prompt', 'Contagem de Avaliações', criterion_name))
            except Exception as e:
                latex_output.append(f"Ocorreu um erro ao analisar o critério {criterion_name}: {e}")

        # =================================================================
        # 2.3 ANÁLISE DE LEGIBILIDADE (ANOVA + TUKEY)
        # =================================================================
        latex_output.append(
            r"""
\section{Análise do Nível de Linguagem}
\label{sec:legibilidade}

A legibilidade das respostas foi avaliada objetivamente por meio de três índices da ferramenta \texttt{textstat} (adaptada para o português):
\begin{itemize}
    \item \textbf{Flesch Reading Ease (FRE):} Pontuações mais altas indicam maior facilidade de leitura (0-100).
    \item \textbf{Flesch-Kincaid Grade Level (FKGL):} Estima os anos de escolaridade formal necessários para compreender o texto.
    \item \textbf{SMOG Index:} Outro índice que estima os anos de escolaridade necessários.
\end{itemize}

Para comparar as médias de legibilidade entre os três tipos de prompt, foi realizado um teste de Análise de Variância (ANOVA). Caso a ANOVA resultasse em um valor-p significativo (< 0.05), um teste \textit{post-hoc} de Tukey HSD foi aplicado para identificar quais pares de prompts possuem médias estatisticamente diferentes.
"""
        )
        try:
            df_respostas_unicas = df_total_consolidado.drop_duplicates(subset=['ID_Pergunta', 'Abordagem']).copy()
            textstat.set_lang('pt_BR')

            for metric in ['flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index']:
                df_respostas_unicas[metric] = df_respostas_unicas['Resposta Gerada'].apply(lambda x: getattr(textstat, metric)(x) if isinstance(x, str) and x else np.nan)

            df_respostas_unicas.dropna(subset=['flesch_reading_ease', 'Abordagem'], inplace=True)

            readability_means = df_respostas_unicas.groupby('Abordagem')[['flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index']].mean()
            readability_means.columns = ['Média Flesch Reading Ease', 'Média Flesch-Kincaid Grade', 'Média SMOG Index']
            latex_output.append(
                readability_means.reset_index().rename(columns={'Abordagem':'Prompt'}).to_latex(index=False, float_format="%.2f", caption='Médias das pontuações de legibilidade por tipo de prompt.', label='tab:legibilidade_means', position='H')
            )

            readability_metrics_map = {
                'flesch_reading_ease': 'Flesch Reading Ease',
                'flesch_kincaid_grade': 'Flesch-Kincaid Grade Level',
                'smog_index': 'SMOG Index'
            }

            for metric_key, metric_name in readability_metrics_map.items():
                latex_output.append(f"\n\\subsection{{Análise para {escape_latex(metric_name)}}}\n")

                df_metric_filtered = df_respostas_unicas.dropna(subset=[metric_key])
                prompt_groups = [df_metric_filtered[df_metric_filtered['Abordagem'] == prompt][metric_key] for prompt in ['Prompt CoT', 'Prompt Direto', 'Prompt Heurístico']]
                valid_groups = [g for g in prompt_groups if not g.empty and len(g) > 1]

                if len(valid_groups) < 2:
                    latex_output.append(f"Não foi possível realizar a análise para {metric_name}, pois não havia dados suficientes em pelo menos dois grupos de prompts.")
                    continue

                stat_levene, p_levene = levene(*valid_groups)
                levene_conclusion = "não foi violado" if p_levene >= 0.05 else "foi violado"
                latex_output.append(f"O teste de Levene para a homogeneidade das variâncias resultou em um valor-p de ${p_levene:.4f}$, indicando que o pressuposto de homogeneidade {levene_conclusion} (p $\\geq$ 0.05).\n")

                f_stat, p_value = f_oneway(*valid_groups)
                dof_between = len(valid_groups) - 1
                dof_within = sum(len(g) for g in valid_groups) - len(valid_groups)

                latex_output.append(f"O teste ANOVA resultou em $F({dof_between}, {dof_within}) = {f_stat:.4f}$, com um valor-p de ${p_value:.4f}$. ")

                if p_value < 0.05:
                    latex_output.append("Este resultado indica que há uma diferença estatisticamente significativa em pelo menos um dos pares de médias de legibilidade. Para identificar quais pares são diferentes, foi realizado o teste de Tukey HSD (Tabela \\ref{{tab:tukey_{metric_key}}}).\n")

                    tukey_results = pairwise_tukeyhsd(endog=df_metric_filtered[metric_key], groups=df_metric_filtered['Abordagem'], alpha=0.05)
                    caption_tukey = f'Resultados do teste post-hoc de Tukey HSD para a métrica {escape_latex(metric_name)}.'
                    label_tukey = f'tab:tukey_{metric_key}'
                    latex_output.append(generate_tukey_latex(tukey_results, caption_tukey, label_tukey))
                else:
                    latex_output.append("Este resultado indica que não há uma diferença estatisticamente significativa nas médias de legibilidade entre os prompts para esta métrica.\n")

                caption_box = f'Distribuição da pontuação de {escape_latex(metric_name)} para cada tipo de prompt.'
                label_box = f"legibilidade_{metric_key}"
                latex_output.append(generate_pgfplot_boxplot(df_metric_filtered, metric_key, f'Legibilidade ({metric_name}) por Tipo de Prompt', 'Tipo de Prompt', f'Pontuação {metric_name}', caption_box, label_box))

        except Exception as e:
            latex_output.append(f"\nOcorreu um erro ao realizar a análise de legibilidade: {traceback.format_exc()}")

        # =================================================================
        # 3. IMPRESSÃO DO CÓDIGO LATEX FINAL
        # =================================================================
        print("\n\n" + "="*60)
        print("    CÓDIGO LATEX GERADO COM SUCESSO (VERSÃO 6.5 - FINAL CORRIGIDO)    ")
        print("="*60)
        print("\nCopie e cole o código abaixo diretamente no seu arquivo .tex.\n")
        print("\n".join(latex_output))

    except Exception as e:
        print(f"\nOcorreu um erro geral durante o processamento: {traceback.format_exc()}")
        print("\nVerifique se os arquivos enviados estão corretos e se as colunas estão no formato esperado.")

--- PASSO 1: UPLOAD E PREPARAÇÃO DOS DADOS ---

Por favor, faça o upload de UM OU MAIS arquivos de avaliação (.xlsx ou .csv).


Saving Instrumento_avaliacao - LARISSA.xlsx to Instrumento_avaliacao - LARISSA (2).xlsx
Saving instrumento_avaliacao_FINAL_GEMINI - fabricia 06.08.xlsx to instrumento_avaliacao_FINAL_GEMINI - fabricia 06.08 (2).xlsx
Saving instrumento_avaliacao_FINAL_GEMINI_Marília.xlsx to instrumento_avaliacao_FINAL_GEMINI_Marília (2).xlsx

Lendo e processando 3 arquivo(s)...
  - Processando 'Instrumento_avaliacao - LARISSA (2).xlsx' como Avaliador(a) 1...
  - Processando 'instrumento_avaliacao_FINAL_GEMINI - fabricia 06.08 (2).xlsx' como Avaliador(a) 2...
  - Processando 'instrumento_avaliacao_FINAL_GEMINI_Marília (2).xlsx' como Avaliador(a) 3...
Dados processados com sucesso.

Iniciando a geração do relatório em LaTeX...


    CÓDIGO LATEX GERADO COM SUCESSO (VERSÃO 6.5 - FINAL CORRIGIDO)    

Copie e cole o código abaixo diretamente no seu arquivo .tex.

% ===================================================================
% INÍCIO DO CAPÍTULO DE RESULTADOS GERADO AUTOMATICAMENTE
% Gerado em: 2025-